# 06_report_export_notify


In [0]:
# Databricks notebook: 06_publish_export_notify
from databricks.sdk import WorkspaceClient
from datetime import datetime
import pandas as pd
import uuid
import os

ws = WorkspaceClient()

approved = spark.sql("""
    SELECT * FROM finance.kyc_gold.report_candidates
    WHERE status='approved'""").toPandas()

if approved.empty:
    print("No approved reports found.")
else:
    for _, report in approved.iterrows():
        print(f"Processing: {report['report_name']}")

        # 1. Publish dashboard
        dash = ws.dashboards.create(name=report['report_name'])
        dashboard_url = f"{ws.config.host}/sql/dashboards/{dash.id}"

        # 2. Export reports
        pdf_path = f"/dbfs/tmp/{report['report_name']}.pdf"
        excel_path = f"/dbfs/tmp/{report['report_name']}.xlsx"

        with open(pdf_path, "w") as f:
            f.write(f"Report: {report['report_name']}")
        pd.DataFrame({"Sample": [1, 2, 3]}).to_excel(excel_path, index=False)

        # 3. Notify (Placeholder print)
        print(f"Email sent to {report['owner']} with {pdf_path} and {excel_path}")

        # 4. Update status
        spark.sql(f"""
            UPDATE finance.kyc_ml.report_candidates
            SET status='published',
                dashboard_id='{dash.id}',
                report_url='{dashboard_url}',
                published_at='{datetime.now()}'
            WHERE id='{report['id']}'
        """)